In [3]:
from  google.colab import drive  
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/algo trading/stocks
### save and load data using pickle
import pickle
def psave(dataframe, fname):
    with open('{}.pkl'.format(fname),'wb') as pickle_file:
        pickle.dump(dataframe,pickle_file)
        
def pload(fname):
  
  with open('{}.pkl'.format(fname),'rb') as pickle_file:
      dataframe=pickle.load(pickle_file)
      return dataframe

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/algo trading/stocks


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import sys

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
sys.path.append("/content/drive/MyDrive/Colab Notebooks/classes")
%reload_ext autoreload
%autoreload 2

from PropertyTest_DataGeneration import Test as t
from PropertyTest_DataGeneration import Gen as g
from ou_process import OUP

In [6]:
train_s=np.datetime64('2013-12-30')
train_e=np.datetime64('2017-12-29')

vali_s=np.datetime64('2018-01-02')
vali_e=np.datetime64('2018-12-31')

test_s=np.datetime64('2019-01-02')
test_e=np.datetime64('2019-12-30')

In [7]:
StockData=pload('StockPool')
totalkeys=StockData.keys()
print(len(totalkeys))

944


In [31]:
#Pair selection
for round in range(1):


  mean1=[]
  CrossTime=[]
  names=[]
  length=len(StockData.keys())
  keys=list(StockData.keys())

  # Computing training set pairs' mean and mean crossing time
  for i in range(length):
    for j in range(i+1,length):
      k1=keys[i]
      k2=keys[j]
      spread=StockData[k1]['Adj Close'][train_s:train_e]-StockData[k2]['Adj Close'][train_s:train_e]
      mean=spread.mean()

      CrossTime.append(t.CrossingTime(mean,spread,train_s,train_e,on=True))
      mean1.append(mean)
      names.append([k1,k2])
  df=pd.DataFrame({'cross':CrossTime,'mean1':mean1,'pair':names})
  df1=df.loc[(df['cross']>=df['cross'].quantile(0.75))].sort_values('cross')
  
  # Cointegration test
  df2=df1.copy()
  for i in list(df1.index):
    k1=df1.loc[i]['pair'][0]
    k2=df1.loc[i]['pair'][1]
    Y=StockData[k1]['Adj Close'].loc[train_s:train_e]
    X=StockData[k2]['Adj Close'].loc[train_s:train_e]
    if t.CointTest(Y,X)==0:
      df2=df2.drop(index=i)


  # Finding beta by fitting spread to OU Process
  betal=[]
  betat=np.linspace(0.05,1,50)
  for i in df2.index:
    k1=df1.loc[i]['pair'][0]
    k2=df1.loc[i]['pair'][1]
    Y=StockData[k1]['Adj Close'].loc[train:validation]
    X=StockData[k2]['Adj Close'].loc[train:validation]
    max=-1000
    beta=0
    for b in betat:
      spread=Y-b*X
      l= OUP.ALL_MLE(spread)[-1]
      if l>max:
        max=l
        beta=b
    betal.append(beta)
  df2['beta']=betal

  # Recomputing the spread using beta
  spreadl=[]
  returnl=[]
  for i in df2.index:
    k1=df1.loc[i]['pair'][0]
    k2=df1.loc[i]['pair'][1]
    Y=StockData[k1]['Adj Close']
    X=StockData[k2]['Adj Close']
    spread=Y-df2['beta'].loc[i]*X
    spreadl.append(spread)
    returnl.append(g.FutureReturnGen(spread))

  df2['spread']=spreadl
  df2['return']=returnl

  

  mean2=[]
  CrossTime2=[]
  for i in df2.index:

    spread=df2['spread'].loc[i].loc[train_s,train_e]
    mean=spread.mean()

    CrossTime2.append(t.CrossingTime(mean,spread,train_s,train_e,on=True))
    mean2.append(mean)
  df2['mean2']=mean2
  df2['cross2']=CrossTime2


  mean3=[]
  CrossTime3=[]
  for i in df2.index:

    spread=df2['spread'].loc[i].loc[vali_s:vali_e]
    mean=spread.mean()

    CrossTime3.append(t.CrossingTime(mean,spread,vali_s,vali_e,on=True))
    mean3.append(mean)

  #selecting pairs having consistent mean and mean crossing time
  mean_diff=np.array(mean2)-np.array(mean3)
  df2['mean3']=mean3
  df2['cross3']=CrossTime3
  df2['meandiff']=mean_diff
  df3=df2.loc[((df2['meandiff']>=df2['meandiff'].quantile(0.25))&(df2['meandiff']<=df2['meandiff'].quantile(0.75)))]
  df3=df3.loc[(df3['cross2']/4 <= df3['cross3'])]

  #final stationary test
  df4=df3.copy()
  for i in df3.index:
    sr=df3.loc[i]['spread'].loc[train_s:vali_e]
    if t.StationarityTest(sr)==0:
      df4=df4.drop(index=i)

  #beta cant be zero
  df4=df4.loc[(df4['beta']!=0)]

  print('Number of pairs:',len(df4.index))
  psave(df4,'finalpairs')

Number of pairs: 463


In [13]:
df=pload('finalpairs')
df

,cross,mean1,pair,beta,spread,return,mean2,cross2,mean3,cross3,meandiff
297369,32,-1.010079,"[PRIM, YNDX]",0.418367,Date 2013-12-02 8.828176 2013-12-03 8.79...,Date 2013-12-02 -0.003229 2013-12-03 -0.02...,12.490825,61,10.049718,17,2.441107
287294,32,-35.081355,"[DMRC, SYNA]",0.088776,Date 2013-12-02 13.501538 2013-12-03 13....,Date 2013-12-02 0.014461 2013-12-03 -0.00...,24.819310,55,23.668305,19,1.151005
404016,32,-2.325309,"[FTEK, MARPS]",0.379592,Date 2013-12-02 4.309182 2013-12-03 4.38...,Date 2013-12-02 0.018453 2013-12-03 0.00...,0.704273,49,0.147969,19,0.556304
247428,32,-1.114856,"[CONN, PTCT]",0.515306,Date 2013-12-02 51.046072 2013-12-03 50....,Date 2013-12-02 -0.014491 2013-12-03 0.00...,11.598712,30,13.956132,15,-2.357420
99273,32,-27.045094,"[AVNW, NTCT]",0.146939,Date 2013-12-02 9.772408 2013-12-03 9.64...,Date 2013-12-02 -0.012856 2013-12-03 -0.03...,2.163562,42,4.206665,16,-2.043104
...,...,...,...,...,...,...,...,...,...,...,...
192718,90,2.527019,"[TCBK, EBTC]",0.786735,Date 2013-12-02 9.799384 2013-12-03 9....,Date 2013-12-02 -0.014110 2013-12-03 -0.00...,7.394992,42,9.676525,11,-2.281533
314459,91,3.059907,"[FDUS, MGIC]",0.961224,Date 2013-12-02 4.596692 2013-12-03 4.48...,Date 2013-12-02 -0.023360 2013-12-03 0.00...,3.302077,91,2.671474,25,0.630603
196213,94,4.778379,"[GBCI, QADB]",0.747959,Date 2013-12-02 12.892984 2013-12-03 12....,Date 2013-12-02 0.003648 2013-12-03 -0.01...,9.759023,64,10.993920,22,-1.234897
444582,95,6.248097,"[QGEN, QADB]",0.689796,Date 2013-12-02 14.429062 2013-12-03 14....,Date 2013-12-02 0.012231 2013-12-03 -0.02...,12.378121,53,10.678263,31,1.699857


In [15]:
#calculating the length of one hypothetical period of each spread, T

import math as mt
t=len(df.iloc[0]['spread'].loc[train_s:train_e])
df['T']=np.ones(l)
for i in range(l):
  num=df['cross2'].iloc[i]/2
  df['T'].iloc[i]=mt.ceil(t/num)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [28]:
#random select 20 for investigation
df2=df.sample(20)

In [25]:
df2

,cross,mean1,pair,beta,spread,return,mean2,cross2,mean3,cross3,meandiff,T
363082,37,-33.884243,"[CTHR, HAYN]",0.050000,Date 2013-12-02 2.929801 2013-12-03 2.79...,Date 2013-12-02 -0.044655 2013-12-03 -0.01...,-0.242134,40,-0.571331,18,0.329197,51.0
99348,33,-17.050050,"[AVNW, MMYT]",0.146939,Date 2013-12-02 11.808979 2013-12-03 11....,Date 2013-12-02 -0.025819 2013-12-03 -0.03...,3.632221,49,3.595694,26,0.036528,42.0
288449,36,-27.130426,"[VOXX, NTCT]",0.224490,Date 2013-12-02 10.296735 2013-12-03 10....,Date 2013-12-02 0.029433 2013-12-03 -0.01...,-0.577103,57,-0.680304,28,0.103202,36.0
216234,35,-2.241410,"[TCBIL, VLGEA]",0.534694,Date 2013-12-02 1.446036 2013-12-03 1....,Date 2013-12-02 -0.154412 2013-12-03 1.13...,8.002588,53,8.540720,14,-0.538132,39.0
314459,91,3.059907,"[FDUS, MGIC]",0.961224,Date 2013-12-02 4.596692 2013-12-03 4.48...,Date 2013-12-02 -0.023360 2013-12-03 0.00...,3.302077,91,2.671474,25,0.630603,23.0
363238,32,-6.491330,"[CTHR, CCLP]",0.185714,Date 2013-12-02 3.362121 2013-12-03 3.20...,Date 2013-12-02 -0.045473 2013-12-03 -0.00...,0.039106,34,0.189550,34,-0.150444,60.0
365248,38,-5.172277,"[TGA, CCLP]",0.282653,Date 2013-12-02 4.670289 2013-12-03 4.67...,Date 2013-12-02 0.001556 2013-12-03 0.06...,0.580727,39,0.724924,11,-0.144197,52.0
288537,40,2.226178,"[VOXX, MARPS]",0.844898,Date 2013-12-02 9.853340 2013-12-03 10....,Date 2013-12-02 0.015442 2013-12-03 -0.00...,2.983573,48,3.017976,43,-0.034403,43.0
439897,34,-1.855255,"[MIND, CCLP]",0.728571,Date 2013-12-02 9.905243 2013-12-03 9.67...,Date 2013-12-02 -0.023199 2013-12-03 -0.03...,0.321557,35,0.012027,15,0.309530,58.0
99073,34,-10.444400,"[AVNW, EPZM]",0.146939,Date 2013-12-02 11.102204 2013-12-03 10....,Date 2013-12-02 -0.028125 2013-12-03 -0.05...,4.602847,44,6.254098,32,-1.651251,46.0


In [27]:
psave(df2,'selectedpairs')